# Build a Deepfake Detection System in 5 Lines

This notebook illustrates how to build a video deepfake detection system from scratch using [Towhee](https://towhee.io/). A deepfake detection system predict videos with probabilities being fake ones. This tutorial will use examples from [Deepfake Detection Challenge](https://www.kaggle.com/competitions/deepfake-detection-challenge).

Using the sample data of different videos, we will build a basic deepfake detection system within 5 lines of code and check the performance using Towhee. At the end, we use [Gradio](https://gradio.app/) to create a showcase that can be played with.

## Preparation

### Install packages

Make sure you have installed required python packages:

| package |
| -- |
| towhee |
| towhee.models |
| dlib |
| facenet-pytorch |
| albumentations |
| timm |
| pytorch_toolbelt |
| tensorboardx |
| tqdm |
| gradio |

In [11]:
! python -m pip install -q towhee towhee.models dlib facenet-pytorch albumentations timm pytorch_toolbelt tensorboardx tqdm gradio

## Build System

Now we are ready to build a deepfake detection system using sample data. We will use the [Combined efficientnet](https://arxiv.org/abs/2107.02612) model to predict the possibilites of being fake ones for input videos. With proper [Towhee operators](https://towhee.io/operators), you don't need to go through video preprocessing & model details. It is very simple to use the [method-chaining style API](https://towhee.readthedocs.io/en/main/index.html) to wrap operators and then apply them to batch inputs.

In [1]:
import towhee

(
    towhee.glob['path']('/home/xuyu/Deepfake/deepfake_detec')
          .deepfake['path', 'scores']()
          .select['path', 'scores']()
          .show()
)

/home/xuyu/anaconda3/envs/miro/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_555_DeepFakeClassifier_tf_efficientnet_b7_ns_0_19
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_40
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36
Predicting 2 videos


path,scores
/home/xuyu/Deepfake/deepfake_det...,"[0.9893,0.9097] len=2"


#### Pipeline Explanation

Here are some details for each line of the assemble pipeline:

- `towhee.deepfake()`: a Towhee operator applying pretrained models to predict the probabilite a video being a fake one. The higher the score, the higher the probability of it being a fake video.[learn more](https://towhee.io/towhee/deepfake)

## Release a Showcase

We've learnt how to build a deepfake detection system. Now it's time to add some interface and release a showcase. Towhee provides `towhee.api()` to wrap the data processing pipeline as a function with `.as_function()`. So we can build a quick demo with this `deepfake_detection_function` with [Gradio](https://gradio.app/).

In [4]:
with towhee.api() as api:
    deepfake_detection_function = (
        api.deepfake()
            .as_function()
    )

In [5]:
import gradio

interface = gradio.Interface(deepfake_detection_function, 
                             inputs = gradio.Textbox(lines=1),
                             outputs = [gradio.Textbox(lines=1)]
                            )

interface.launch(inline=True, share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://26947.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f60093e9990>,
 'http://127.0.0.1:7860/',
 'https://26947.gradio.app')

loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_555_DeepFakeClassifier_tf_efficientnet_b7_ns_0_19
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_888_DeepFakeClassifier_tf_efficientnet_b7_ns_0_40
loading state dict /home/xuyu/.towhee/hub/towhee/deepfake/main/weights/final_111_DeepFakeClassifier_tf_efficientnet_b7_ns_0_36
Predicting 2 videos


<img src='deepfake.jpg' alt='deepfake_detection_demo' width=700px/>